<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Insurance Analysis Using Temporal</b>
</header>


<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>

<p style = 'font-size:18px;font-family:Arial'>
Applications require to design and build databases dependant on time or data changes over time.  Building such applications is possible but becomes easier with Temporal
   
</p>  

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>WHY TEMPORAL? </b> </p>

<p style = 'font-size:18px;font-family:Arial'>    
Things You No Longer Have To Do in the Time-Aware Teradata Database
Data Maintenance : </p>
    
<li style = 'font-size:16px;font-family:Arial'> Update effective dates in data.</li></<li style = 'font-size:16px;font-family:Arial'> <li style = 'font-size:16px;font-family:Arial'> Specify “transaction time” timestamp when data are entered or updated. </li>
<li style = 'font-size:16px;font-family:Arial'> Insert new data rows for updates. </li>     
<li style = 'font-size:16px;font-family:Arial'> Manage temporal history storage space. </li> 
  

 
<p style = 'font-size:18px;font-family:Arial'> Query and Application Development : </p>


<li style = 'font-size:16px;font-family:Arial'> Write complex period based condition clauses. </li>
<li style = 'font-size:16px;font-family:Arial'> Add effective date qualifications to “current queries.” </li>
<li style = 'font-size:16px;font-family:Arial'> Train users and application developers in effective date columns and processing for “AS OF” query development.</li>
<li style = 'font-size:16px;font-family:Arial'> Save old reports or database snapshots to facilitate looking backfor analysis, compliance reporting, or audit trails.</li>

<p style = 'font-size:16px;font-family:Arial'>Here we are considering the example of Insurance company storing data for various policies whose validity, conditions , premium amount and many other factors will change over Time period.</p>
    
    
 

   
    
    
    
<p style = 'font-size:18px;font-family:Arial'>Some scenarios that can be handled using temporal features are as below 
    
<li style = 'font-size:16px;font-family:Arial'>    Consider the scenario where there is a hail storm occuring on a particular date. The As of date feature can be used to check the validity of the policy and than the incident date can be used to check which claims fall in the validity period of the policies.</li>
    
<li style = 'font-size:16px;font-family:Arial'>    Another scenario is that the customer has sold the vehicle and the closure of the policy happened a week late as the customer informed late about the sale , so based on the transaction time the validity of the policies will change. </li>



    
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Accessing the Data </b></p>
<p style = 'font-size:16px;font-family:Arial'>These demos will work either with foreign tables accessed from Cloud Storage via NOS or you may import the tables to your machine. If you import data for multiple demos, you may need to use the Data Dictionary "Manage Your Space" routine to cleanup tables you no longer need. 
    
<p style = 'font-size:16px;font-family:Arial'>Use the link below to access the 2 options for using data from the data dictionary notebook:    
    
    
[Click Here to get data for this notebook](../Data_Dictionary/Data_Dictionary.ipynb#TRNG_InsuranceClaims)    
    
    
[Click Here to Manage Your Space](../Data_Dictionary/Data_Dictionary.ipynb#Manage_Your_Space)
    

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Let's start by connecting to the Teradata system </b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password </p>

In [ ]:
%connect local

<p style = 'font-size:16px;font-family:Arial'>To understand how Temporal can be helpful we will consider a simple example where we need to check policies which were valid on 31st December2020 and had a premium amount less than 1000. for a normal table with start and end date columns the query will be as below :</p>

In [ ]:
SELECT Policy_ID, Customer_ID, Policy_premium_Amt,policy_state,Policy_start_date, policy_end_date 
FROM TRNG_InsuranceClaims.Policy_Details_Org
where policy_start_date <= '2020-12-31'
and policy_end_date >= '2020-12-31'
and Policy_premium_Amt < 1000

<p style = 'font-size:16px;font-family:Arial'>The same query with Temporal table would be as below :</p>

In [ ]:
VALIDTIME AS OF Date'2020-12-31'
SELECT Policy_ID, Customer_ID, Policy_premium_Amt,policy_state,Policy_Validity FROM TRNG_InsuranceClaims.Policy_Details
where Policy_premium_Amt < 1000


<p style = 'font-size:18px;font-family:Arial'><b> Now let's check how temporal can help simplify queries in a complex scenario:</b></p>

<p style = 'font-size:16px;font-family:Arial'>Consider scenario where we have to check all valid policies whose validity period is similar to the Policy of the Mercedes -ML350 in the state IL. In a table without the temporal columns the query would have to collect data for various scenarios where the policy_start_date and policy_end_date for other policies lie in the range of the policy_start_date and policy_end_date of the policy of Mercedes-ML350 in state of IL. So the query will have to do a UNION ALL with different date checks for checking the start and end date of the policies looks like the below : </p>

In [ ]:
select OthPol.Policy_id, OthPol.Customer_id, OthPol.auto_make , OthPol.Policy_start_date,OthPol.Policy_end_date
from TRNG_InsuranceClaims.policy_details_org MercPol --- "Table for getting details of the Mercedes-ML350"
, TRNG_InsuranceClaims.policy_details_org OthPol --- "Table Instance for creating self join to get details of other policies"
where MercPol.auto_make = 'Mercedes'
and MercPol.policy_state = 'IL'
and MercPol.auto_model = 'ML350'
and MercPol.policy_state = OthPol.Policy_state
AND MercPol.policy_start_date <= OthPol.policy_start_date
AND MercPol.policy_end_date > OthPol.policy_start_date
AND MercPol.policy_end_date <= OthPol.policy_end_date
UNION ALL
select OthPol.Policy_id, OthPol.Customer_id, OthPol.auto_make , OthPol.Policy_start_date,OthPol.Policy_end_date
from TRNG_InsuranceClaims.policy_details_org MercPol
, TRNG_InsuranceClaims.policy_details_org OthPol
where MercPol.auto_make = 'Mercedes'
and MercPol.policy_state = 'IL'
and MercPol.auto_model = 'ML350'
and MercPol.policy_state = OthPol.Policy_state
AND MercPol.policy_start_date <= OthPol.policy_start_date
AND MercPol.policy_end_date >= OthPol.policy_end_date
UNION ALL
select OthPol.Policy_id, OthPol.Customer_id, OthPol.auto_make , OthPol.Policy_start_date,OthPol.Policy_end_date
from TRNG_InsuranceClaims.policy_details_org MercPol
, TRNG_InsuranceClaims.policy_details_org OthPol
where MercPol.auto_make = 'Mercedes'
and MercPol.policy_state = 'IL'
and MercPol.auto_model = 'ML350'
and MercPol.policy_state = OthPol.Policy_state
AND MercPol.policy_start_date >= OthPol.policy_start_date
AND MercPol.policy_end_date <= OthPol.policy_end_date
UNION ALL
select OthPol.Policy_id, OthPol.Customer_id, OthPol.auto_make , OthPol.Policy_start_date,OthPol.Policy_end_date
from TRNG_InsuranceClaims.policy_details_org MercPol
, TRNG_InsuranceClaims.policy_details_org OthPol
where MercPol.auto_make = 'Mercedes'
and MercPol.policy_state = 'IL'
and MercPol.auto_model = 'ML350'
and MercPol.policy_state = OthPol.Policy_state
AND MercPol.policy_start_date >= OthPol.policy_start_date
AND MercPol.policy_start_date < OthPol.policy_end_date
AND MercPol.policy_end_date >= OthPol.policy_end_date;

<p style = 'font-size:16px;font-family:Arial'>Use the Temporal ValidTime function to display similar results as the query written above on table without Period datatype. </p>

In [ ]:
Validtime select OthPol.Policy_id, OthPol.Customer_id, OthPol.auto_make 
from TRNG_InsuranceClaims.policy_details MercPol , TRNG_InsuranceClaims.policy_details OthPol
where MercPol.auto_make = 'Mercedes'
and MercPol.policy_state = 'IL'
and MercPol.auto_model = 'ML350'
and MercPol.policy_state = OthPol.Policy_state
;

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>Comparison for Queries with and without Temporal </b></p>

<table>
<tr>
<td style="color:black;font-size:18px;font-weight:bold">WITH TEMPORAL</td>
<td style="color:black;font-size:18px;font-weight:bold">WITHOUT TEMPORAL</td> 
</tr>

<tr>
<td>Validtime select b.Policy_id, b.Customer_id, b.auto_make from policy_details a , policy_details b where a.auto_make = 'Mercedes' and a.policy_state = 'IL' and a.auto_model = 'ML350' and a.policy_state = b.Policy_state; </td>
<td>select b.Policy_id, b.Customer_id, b.auto_make , b.Policy_start_date,b.Policy_end_date from policy_details_org_extrapolate a , policy_details_org_extrapolate b where a.auto_make = 'Mercedes' and a.policy_state = 'IL' and a.auto_model = 'ML350' and a.policy_state = b.Policy_state AND a.policy_start_date b.policy_start_date AND a.policy_end_date    UNION ALL 
select b.Policy_id, b.Customer_id, b.auto_make , b.Policy_start_date,b.Policy_end_date from policy_details_org_extrapolate a , policy_details_org_extrapolate b where a.auto_make = 'Mercedes' and a.policy_state = 'IL' and a.auto_model = 'ML350' and a.policy_state = b.Policy_state AND a.policy_start_date <= b.policy_start_date AND a.policy_end_date >= b.policy_end_date

   UNION ALL 
select b.Policy_id, b.Customer_id, b.auto_make , b.Policy_start_date,b.Policy_end_date from policy_details_org_extrapolate a , policy_details_org_extrapolate b where a.auto_make = 'Mercedes' and a.policy_state = 'IL' and a.auto_model = 'ML350' and a.policy_state = b.Policy_state AND a.policy_start_date >= b.policy_start_date AND a.policy_end_date <= b.policy_end_date

   UNION ALL
select b.Policy_id, b.Customer_id, b.auto_make , b.Policy_start_date,b.Policy_end_date from policy_details_org_extrapolate a , policy_details_org_extrapolate b where a.auto_make = 'Mercedes' and a.policy_state = 'IL' and a.auto_model = 'ML350' and a.policy_state = b.Policy_state AND a.policy_start_date >= b.policy_start_date AND a.policy_start_date < b.policy_end_date AND a.policy_end_date >= b.policy_end_date;</td>
</tr>    
</table>    
 

 <p style = 'font-size:18px;font-family:Arial;'> <b>Performance Comparison for above queries </b> </p>

 <img src="images/PerformanceMetrics.png" style="width:1000px;height:200px;">

<p style = 'font-size:16px;font-family:Arial'>Comparing the Validity and VALIDTIME columns demonstrates that the   
    valid-time period of the result set is the intersection of the valid-time period in the original Policy table Validity 
    column with the Period specified in the sequenced select statement.</p>


In [ ]:
SEQUENCED VALIDTIME PERIOD '(2009-01-01, 2009-12-31)'
SELECT Policy_ID, Customer_ID, Policy_Validity FROM (
SEQUENCED VALIDTIME SELECT Policy_details.*
FROM TRNG_InsuranceClaims.Policy_Details where policy_state = 'IN') AS my_derived_table;


<p style = 'font-size:16px;font-family:Arial'>Aggregations can be used for analysis of the data based on the validity period. 
Comparisons of data between different validity periods can be achieved with Temporal. Like the minimum and maximum premium amounts for a particular state based on the model of the cars.</p>



In [ ]:
SEQUENCED VALIDTIME Period (date '2011-01-01', date '2020-12-31')
SELECT auto_make,
min(Policy_premium_Amt) as MinPremium,
max(Policy_premium_Amt) as MaxPremium
FROM TRNG_InsuranceClaims.Policy_Details
where policy_state = 'OH'
GROUP BY 1
ORDER BY VALIDTIME;



<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>Temporal concepts </b></p>

<p style = 'font-size:18px;font-family:Arial'> <b> Time line for various Policies :-    Analysis can be done for </b> </p>
    
   
<li style = 'font-size:16px;font-family:Arial'>Policies valid during a particular period.</li>
    
<li style = 'font-size:16px;font-family:Arial'>Validity period of various policies.</li>
    
<li style = 'font-size:16px;font-family:Arial'>Premium amount collected during a particular period </li>

<p style = 'font-size:18px;font-family:Arial'>Using the As Of concept to verify the active policies on a particular date</p>

<p style = 'font-size:16px;font-family:Arial'>On a given date, there was a hail storm and we want to determine the count of policies in effect on that date
The scenario mentioned here can be solved by the below query using the As of Date feature of Temporal tables   
 </p>


<img src="images/TemporalTimePeriod.png" style="width:600px;height:400px;">



In [ ]:
VALIDTIME AS OF Date'2022-02-15'
SELECT Policy_ID, Customer_ID, Policy_premium_Amt,Policy_Validity FROM TRNG_InsuranceClaims.Policy_Details
where policy_state = 'IL'

In [ ]:
Select count(*) as "no of Policies" from (VALIDTIME AS OF Date'2022-02-15'
SELECT Policy_ID, Customer_ID, Policy_premium_Amt,Policy_Validity FROM TRNG_InsuranceClaims.Policy_Details 
    where policy_state = 'IL') as DerivTbl

<p style = 'font-size:16px;font-family:Arial'>Consider scenario where we need to check that for a particular period(here for year 2021) how many policies are valid for each auto make type and then compare the no of policies which are valid in the current period. This helps us in the comparision in the increase or decrease of policies for the current year as compared to the past year of validity
 </p>


In [ ]:
Select auto_make,count(*) as "no of Policies" from (VALIDTIME Period (date'2021-01-01',date'2021-12-31')
SELECT Policy_ID, Customer_ID, Policy_premium_Amt,auto_make FROM TRNG_InsuranceClaims.Policy_Details ) as DerivTbl 
group by 1;

In [ ]:
%chart x=auto_make, y="no of Policies", title= Valid Number of Policies last year 

<p style = 'font-size:16px;font-family:Arial'>Now we check how many policies are still valid for each type using the current functionality
 </p>


In [ ]:
Select auto_make,count(*) as "no of Policies" from (current VALIDTIME 
SELECT Policy_ID, Customer_ID, Policy_premium_Amt,auto_make FROM TRNG_InsuranceClaims.Policy_Details ) as DerivTbl
group by 1;

In [ ]:
%chart x=auto_make, y="no of Policies", title= Current Valid Number of Policies  

<p style = 'font-size:16px;font-family:Arial'>In this scenario we are considering that there are a number of claims raised against policies on various incident dates. Below queries help to identify which claims made are in the validity period and which are not and which policies are valid at the time of the incidents.There are total 69 claims in the Insurance Claims tables and based on the incident date we can verify which claims are valid and which are not valid.
 </p>


In [ ]:
SELECT * from TRNG_InsuranceClaims.Insurance_Claims_Details;


<p style = 'font-size:16px;font-family:Arial'>Based on the Claims made , we need to validate which claims are valid on the incident date mentioned. Below query helps in the analysis. Only the rows that qualify for the claims are displayed.
 </p>


In [ ]:
SELECT  'Valid Claims' as ClaimType , Policy.customer_id,Policy.Policy_id, Insurance_Claims_Details.Claim_Id, Claim_amount
FROM (current VALIDTIME SELECT Policy_Details.* FROM TRNG_InsuranceClaims.Policy_Details )Policy
INNER JOIN TRNG_InsuranceClaims.Insurance_Claims_Details ON Policy.Policy_id = Insurance_Claims_Details.Policy_id
WHERE Policy.Policy_Validity CONTAINS Insurance_Claims_Details.Incident_date
;


<p style = 'font-size:16px;font-family:Arial'>Below query displays the Invalid rows from Claims table
 </p>


In [ ]:
SELECT customer_id,Policy.Policy_id, Claim_Id, Claim_amount
from TRNG_InsuranceClaims.Insurance_Claims_Details INNER JOIN TRNG_InsuranceClaims.Policy_Details Policy
ON Policy.Policy_id = Insurance_Claims_Details.Policy_id
MINUS
SELECT Policy.customer_id,Policy.Policy_id, Insurance_Claims_Details.Claim_Id, Claim_amount
FROM (current VALIDTIME SELECT Policy_Details.* FROM TRNG_InsuranceClaims.Policy_Details )Policy
INNER JOIN TRNG_InsuranceClaims.Insurance_Claims_Details ON Policy.Policy_id = Insurance_Claims_Details.Policy_id
WHERE Policy.Policy_Validity CONTAINS Insurance_Claims_Details.Incident_date;


<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>Explaining the Temporal Concepts of Inserts /Deletes </b></p> 

<p style = 'font-size:16px;font-family:Arial'>Customer 1 has policy 1 with Validity Period . In between the validity period the customer decides to change some factors which causes the premium to increase , so there will be a new addition which will insert a new row having validity period from changed date to the end of period. </p>
    
    
    
<p style = 'font-size:16px;font-family:Arial'>Customer 2 has Policy 1 with validity period. In between the company decides to increase the premium which also increases the validity period.
 </p>


<img src="images/Temporal_Inserts_Updates.png" style="width:600px;height:400px;">



<p style = 'font-size:18px;font-family:Arial'>Inserting data into temporal table is simple for current validtime. </p>

In [ ]:
current validtime Insert TRNG_InsuranceClaims_DB.TRNG_InsuranceClaims_Policy_Det 
values (203914, 2316038,'OH'  ,1200,'Audi','A3');

<p style = 'font-size:18px;font-family:Arial'>Inserting history rows will need Temporal Qualifier. </p>

In [ ]:
sequenced validtime Insert TRNG_InsuranceClaims_DB.TRNG_InsuranceClaims_Policy_Det 
values (203914, 2316038,'OH'  ,1050,'Audi','A3' ,PERIOD(date'2002-06-10', date'2012-06-09'));
sequenced validtime Insert TRNG_InsuranceClaims_DB.TRNG_InsuranceClaims_Policy_Det 
values (203914, 2316038,'OH'  ,1150,'Audi','A3' ,PERIOD(date'2012-06-10', date'2022-06-27'));


In [ ]:
sequenced validtime select * from TRNG_InsuranceClaims.Policy_Details where customer_id = 2316038 order by validtime;

<p style = 'font-size:16px;font-family:Arial'>Without Temporal Qualifier the select will display only current row. </p>

In [ ]:
----SET SESSION CURRENT VALIDTIME;
 

In [ ]:
select * from TRNG_InsuranceClaims.Policy_Details where customer_id = 2316038 order by Policy_validity;

<p style = 'font-size:16px;font-family:Arial'>When period is specified , Delete will only delete rows for that specific period (highlighted row in the above dataset) and the validity period for rows will change to the end of period specified in the delete. </p> 

<p style = 'font-size:16px;font-family:Arial'>Display rows for Specific Valid Period.</p>

In [ ]:
SEQUENCED VALIDTIME PERIOD '(1990-01-01, 1990-03-31)' SELECT * 
FROM TRNG_InsuranceClaims.Policy_details where policy_state = 'IN' order by validtime;


<p style = 'font-size:16px;font-family:Arial'>Delete rows for Specific Valid Period will change the validity period to the end date specified in delete. </p>

In [ ]:
SEQUENCED VALIDTIME PERIOD '(1990-01-01, 1990-01-30)' DELETE
FROM TRNG_InsuranceClaims_DB.TRNG_InsuranceClaims_Policy_Det where policy_state = 'IN';


In [ ]:
SEQUENCED VALIDTIME PERIOD '(1990-01-01, 1990-03-31)' SELECT * 
FROM TRNG_InsuranceClaims_DB.TRNG_InsuranceClaims_Policy_Det where policy_state = 'IN' order by validtime;

<p style = 'font-size:18px;font-family:Arial'> <b>Usage of Bi Temporal Table with ValidTime and TransactionTime Columns.</b> </p>

<p style = 'font-size:16px;font-family:Arial'>Select data for a particular range of data temporal features need to be added to the select. </b>

In [ ]:
sequenced validtime and transactiontime as of current_timestamp
sel * from TRNG_InsuranceClaims.Policy_Bitemp

<p style = 'font-size:16px;font-family:Arial'>Updating the Policy amount will also change the transaction time for the policies. </b>

In [ ]:
UPDATE TRNG_InsuranceClaims_DB.TRNG_InsuranceClaims_Policy_Bitemp
SET Policy_premium_Amt = Policy_premium_Amt + 10
WHERE policy_id in (
361834
,602704
,208303
,524215)


In [ ]:
sequenced validtime and current transactiontime
sel * from TRNG_InsuranceClaims.Policy_Bitemp

<p style = 'font-size:16px;font-family:Arial'>Consider the scenario when the validity changes a week later than the actual date which causes changes in the current validity </p>

In [ ]:
Current Validtime
UPDATE TRNG_InsuranceClaims_DB.TRNG_InsuranceClaims_Policy_Bitemp
SET Policy_premium_Amt =  Policy_premium_Amt
WHERE policy_id in (686821
)


In [ ]:
sequenced validtime and current transactiontime
sel * from TRNG_InsuranceClaims.Policy_Bitemp

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>CleanUp </b></p>

In [ ]:
drop view TRNG_InsuranceClaims.Policy_Details_org;

In [ ]:
drop view TRNG_InsuranceClaims.Policy_Details;

In [ ]:
drop view TRNG_InsuranceClaims.Insurance_Claims_Details;

In [ ]:
drop view TRNG_InsuranceClaims.Policy_Bitemp;

In [ ]:
drop table TRNG_InsuranceClaims_DB.TRNG_InsuranceClaims_Policy_Det;

In [ ]:
drop table TRNG_InsuranceClaims_DB.TRNG_InsuranceClaims_Details;

In [ ]:
drop table TRNG_InsuranceClaims_DB.TRNG_InsuranceClaims_Policy_Bitemp;

In [ ]:
drop table gs_tables_db.TRNG_InsuranceClaims_PolicyDet_org;

In [ ]:
drop database TRNG_InsuranceClaims;

In [ ]:
drop database TRNG_InsuranceClaims_DB;

</h4>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Reference Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Information about Temporal datatype can be found at: <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Temporal-Table-Support/July-2021'>https://docs.teradata.com/r/Teradata-VantageTM-Temporal-Table-Support/July-2021</a></li>
</ul>



<footer style="paddinga:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>